# Tablas Pivotantes con Pandas

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
data_path = Path('pivot_data.csv')

In [ ]:
#Una tabla pivotante consiste en organizar los datos de una tabla en una matriz, donde las filas 
#representan una variable y las columnas otra variable.
#En este caso, la tabla pivotante se realiza con la función pivot_table de pandas.
df = pd.read_csv(data_path, sep=',')
df.head()

,date,variable,value,variable2
0,2000-01-03,A,-0.272917,1
1,2000-01-04,A,0.098333,2
2,2000-01-05,A,0.273676,3
3,2000-01-03,B,-0.585902,1
4,2000-01-04,B,0.959133,2


In [11]:
#En este caso vamos a hacer una tabla pivotante donde las filas representan la fecha y las columnas
#representan las variables. Por lo que el índice será la fecha, las columnas serán las variables y los
#valores serán los valores de las variables.
dfp = df.pivot(index='date', columns='variable', values='value')
dfp

variable,A,B,C,D
date,,,,
2000-01-03,-0.272917,-0.585902,-0.164357,-1.242624
2000-01-04,0.098333,0.959133,-1.049870,-0.532904
2000-01-05,0.273676,1.447109,-0.097525,0.348482


In [14]:
#También se podría poner como un multi-índice, donde el índice sea la fecha y la variable.
dfp = df.pivot(index=['date', 'variable'], columns='variable2' ,values='value')
print(dfp.index)
dfp


MultiIndex([('2000-01-03', 'A'),
            ('2000-01-03', 'B'),
            ('2000-01-03', 'C'),
            ('2000-01-03', 'D'),
            ('2000-01-04', 'A'),
            ('2000-01-04', 'B'),
            ('2000-01-04', 'C'),
            ('2000-01-04', 'D'),
            ('2000-01-05', 'A'),
            ('2000-01-05', 'B'),
            ('2000-01-05', 'C'),
            ('2000-01-05', 'D')],
           names=['date', 'variable'])


variable2                   1         2         3
date       variable                              
2000-01-03 A        -0.272917       NaN       NaN
           B        -0.585902       NaN       NaN
           C        -0.164357       NaN       NaN
           D        -1.242624       NaN       NaN
2000-01-04 A              NaN  0.098333       NaN
           B              NaN  0.959133       NaN
           C              NaN -1.049870       NaN
           D              NaN -0.532904       NaN
2000-01-05 A              NaN       NaN  0.273676
           B              NaN       NaN  1.447109
           C              NaN       NaN -0.097525
           D              NaN       NaN  0.348482

In [17]:
#Stack y Unstack.
#La función stack() permite mover las columnas a los índices.
#La función unstack() permite mover los índices a las columnas.
stack = dfp.stack().to_frame()
print(stack.index)
stack #Se ha movido la columna variable2 a los índices. 

MultiIndex([('2000-01-03', 'A', 1),
            ('2000-01-03', 'B', 1),
            ('2000-01-03', 'C', 1),
            ('2000-01-03', 'D', 1),
            ('2000-01-04', 'A', 2),
            ('2000-01-04', 'B', 2),
            ('2000-01-04', 'C', 2),
            ('2000-01-04', 'D', 2),
            ('2000-01-05', 'A', 3),
            ('2000-01-05', 'B', 3),
            ('2000-01-05', 'C', 3),
            ('2000-01-05', 'D', 3)],
           names=['date', 'variable', 'variable2'])


0
date       variable variable2          
2000-01-03 A        1         -0.272917
           B        1         -0.585902
           C        1         -0.164357
           D        1         -1.242624
2000-01-04 A        2          0.098333
           B        2          0.959133
           C        2         -1.049870
           D        2         -0.532904
2000-01-05 A        3          0.273676
           B        3          1.447109
           C        3         -0.097525
           D        3          0.348482

In [18]:
unstack = stack.unstack(level=1)
unstack #Se ha movido el índice variable a las columnas. Level indica el nivel del índice que se quiere mover a las columnas.

0                              
variable                     A         B         C         D
date       variable2                                        
2000-01-03 1         -0.272917 -0.585902 -0.164357 -1.242624
2000-01-04 2          0.098333  0.959133 -1.049870 -0.532904
2000-01-05 3          0.273676  1.447109 -0.097525  0.348482

In [20]:
#Melt. La función melt() permite transformar una tabla pivotante en una tabla larga.
data_cal = pd.DataFrame({'nombre' : ['Juan', 'Ana', 'Pedro'],
                        'apellido' : ['Perez', 'Gomez', 'Gonzalez'],
                        'nota' : [7, 8.9, 9.3],
                        'num_asignaturas' : [5, 6, 7]})
data_cal

,nombre,apellido,nota,num_asignaturas
0,Juan,Perez,7.0,5
1,Ana,Gomez,8.9,6
2,Pedro,Gonzalez,9.3,7


In [ ]:
#Si hacemos un melt() lo que vamos a hacer es tener una fila por cada valor de la tabla. En base a la variable
#que se le pase como id_vars, se van a repetir los valores de las otras variables.
data_cal_melt = data_cal.melt(id_vars=['nombre', 'apellido'], value_vars=['nota', 'num_asignaturas'], var_name='variable', value_name='value')
data_cal_melt
# En este caso, Juan va a tener dos filas, una con la nota y otra con el número de asignaturas. Lo mismo para Ana y Pedro.

,nombre,apellido,variable,value
0,Juan,Perez,nota,7.0
1,Ana,Gomez,nota,8.9
2,Pedro,Gonzalez,nota,9.3
3,Juan,Perez,num_asignaturas,5.0
4,Ana,Gomez,num_asignaturas,6.0
5,Pedro,Gonzalez,num_asignaturas,7.0


In [25]:
#También tenemos la función pivot_table() que permite realizar funciones de agregación en la tabla pivotante.
#En este caso, vamos a hacer la mediana de las notas de los alumnos.
pd.pivot_table(data = data_cal, index=['nombre', 'apellido'], values=['nota'], aggfunc='median')
#En este caso, la mediana de las notas de Juan es 7, la de Ana es 8.9 y la de Pedro es 9.3. Dado que solo hay una nota por alumno.


,,nota
nombre,apellido,
Ana,Gomez,8.9
Juan,Perez,7.0
Pedro,Gonzalez,9.3
